In [21]:
%%writefile module/myApp3.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter

st.set_page_config(layout="wide")
st.title("Screen 2: Retention & Magic Moment (Survival-based)")

# -----------------------------
# 1. 데이터 로드
# -----------------------------
df = pd.read_csv("data/Subscription_Service_Churn_Dataset.csv")

# AccountAge = 총 사용 개월 수
df['tenure_months'] = df['AccountAge']


# 장기 유지 여부 (6개월 이상)
df['long_term'] = df['tenure_months'] >= 6

# -----------------------------
# 2. Survival Cohort Heatmap
# (AccountAge 기반)
# -----------------------------

# 가입 연차 코호트 (0~3, 4~6, 7~12, 12+)
df['cohort_group'] = pd.cut(
    df['tenure_months'],
    bins=[0,3,6,12,24,100],
    labels=['0-3m','4-6m','7-12m','1-2y','2y+']
)

cohort_data = (
    df.groupby(['cohort_group','tenure_months'])['CustomerID']
    .nunique()
    .reset_index()
)

cohort_pivot = cohort_data.pivot(
    index='cohort_group',
    columns='tenure_months',
    values='CustomerID'
)

cohort_size = cohort_pivot.iloc[:,0]
retention = cohort_pivot.divide(cohort_size, axis=0)
retention = retention.iloc[::-1]
retention_smooth = gaussian_filter(retention.values, sigma=1)

fig1, ax1 = plt.subplots(figsize=(14,7))
sns.heatmap(
    retention,
    cmap="viridis",
    vmin=0,
    vmax=1,
    ax=ax1,
    cbar_kws={'label': 'Survival Probability'}
)
ax1.set_title(" Survival Heatmap (AccountAge based)")
ax1.set_xlabel("Months Since Signup")
ax1.set_ylabel("Account Age Cohort")

ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)

xticks = np.arange(0, retention.shape[1], 6)
ax1.set_xticks(xticks)
ax1.set_xticklabels(xticks)

st.pyplot(fig1)

# -----------------------------
# 3. Magic Moment Scatter
# -----------------------------
df['weekly_hours'] = df['ViewingHoursPerWeek']
df['tenure_months'] = df['AccountAge']

fig2, ax2 = plt.subplots(figsize=(10,6))
sns.scatterplot(
    data=df,
    x='weekly_hours',
    y='tenure_months',
    hue='long_term',
    alpha=0.6,
    ax=ax2
)

ax2.axvline(10, linestyle='--')
ax2.axhline(6, linestyle='--')
ax2.set_title("Magic Moment: Weekly Viewing vs Survival")
ax2.set_xlabel("Viewing Hours Per Week")
ax2.set_ylabel("Account Age (Months)")

st.pyplot(fig2)

# -----------------------------
# 4. Magic Moment 정량화
# -----------------------------
# Engagement Score 생성
df['engagement_score'] = (
    df['ViewingHoursPerWeek'] * 0.4 +
    df['ContentDownloadsPerMonth'] * 0.3 +
    df['WatchlistSize'] * 0.2 +
    df['UserRating'] * 0.1
)

# 상위 25%를 Magic Group으로 정의
threshold = df['engagement_score'].quantile(0.75)

df['magic_user'] = df['engagement_score'] >= threshold
df['long_term'] = df['AccountAge'] >= 6

baseline = df['long_term'].mean()
magic_prob = df[df['magic_user']]['long_term'].mean()
lift = magic_prob - baseline

threshold = 10
magic_users = df[df['weekly_hours'] >= threshold]

prob = magic_users['long_term'].mean()
baseline = df['long_term'].mean()
lift = prob - baseline

st.metric("전체 평균 6개월 유지율", f"{baseline*100:.1f}%")
st.metric("주 10시간 이상 시청 유지율", f"{prob*100:.1f}%")
st.metric("Lift 효과", f"+{lift*100:.1f}%p")

st.markdown(f"""
### Insight 

주당 {threshold}시간 이상 콘텐츠를 시청하는 이용자는  
전체 평균 대비 **{lift*100:.1f}%p 확률**로  
6개월 이상 장기 사용자로 유지되었다.  

이는 **콘텐츠 소비 강도(Viewing Intensity)** 가  
구독 서비스의 생존기간을 설명하는  
핵심 행동 지표(Magic Moment)임을 시사한다.
""")


Overwriting module/myApp3.py


In [10]:
print(df.columns)

Index(['AccountAge', 'MonthlyCharges', 'TotalCharges', 'SubscriptionType',
       'PaymentMethod', 'PaperlessBilling', 'ContentType', 'MultiDeviceAccess',
       'DeviceRegistered', 'ViewingHoursPerWeek', 'AverageViewingDuration',
       'ContentDownloadsPerMonth', 'GenrePreference', 'UserRating',
       'SupportTicketsPerMonth', 'Gender', 'WatchlistSize', 'ParentalControl',
       'SubtitlesEnabled', 'CustomerID', 'Churn'],
      dtype='object')


In [23]:
%%writefile module/myApp3.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter

st.set_page_config(layout="wide")
st.title("Screen 2: \n  Retention & Magic Moment (Survival-based)")

# -----------------------------
# 1. 데이터 로드
# -----------------------------
df = pd.read_csv("data/Subscription_Service_Churn_Dataset.csv")

# AccountAge = 총 사용 개월 수
df['tenure_months'] = df['AccountAge']


# 장기 유지 여부 (6개월 이상)
df['long_term'] = df['tenure_months'] >= 6

# -----------------------------
# 2. Survival Cohort Heatmap
# (AccountAge 기반)
# -----------------------------

# 가입 연차 코호트 (0~3, 4~6, 7~12, 12+)
df['cohort_group'] = pd.cut(
    df['tenure_months'],
    bins=[0,3,6,12,24,100],
    labels=['0-3m','4-6m','7-12m','1-2y','2y+']
)

cohort_data = (
    df.groupby(['cohort_group','tenure_months'])['CustomerID']
    .nunique()
    .reset_index()
)

cohort_pivot = cohort_data.pivot(
    index='cohort_group',
    columns='tenure_months',
    values='CustomerID'
)

cohort_size = cohort_pivot.iloc[:,0]
retention = cohort_pivot.divide(cohort_size, axis=0)
retention = retention.iloc[::-1]
retention_smooth = gaussian_filter(retention.values, sigma=1)

fig1, ax1 = plt.subplots(figsize=(14,7))
sns.heatmap(
    retention,
    cmap="viridis",
    vmin=0,
    vmax=1,
    ax=ax1,
    cbar_kws={'label': 'Survival Probability'}
)
ax1.set_title(" Survival Heatmap (AccountAge based)")
ax1.set_xlabel("Months Since Signup")
ax1.set_ylabel("Account Age Cohort")

ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)

xticks = np.arange(0, retention.shape[1], 6)
ax1.set_xticks(xticks)
ax1.set_xticklabels(xticks)

st.pyplot(fig1)

# -----------------------------
# 3. Magic Moment Scatter
# -----------------------------
df['weekly_hours'] = df['ViewingHoursPerWeek']
df['tenure_months'] = df['AccountAge']

fig2, ax2 = plt.subplots(figsize=(10,6))
sns.scatterplot(
    data=df,
    x='weekly_hours',
    y='tenure_months',
    hue='long_term',
    alpha=0.6,
    ax=ax2
)

ax2.axvline(10, linestyle='--')
ax2.axhline(6, linestyle='--')
ax2.set_title("Magic Moment: Weekly Viewing vs Survival")
ax2.set_xlabel("Viewing Hours Per Week")
ax2.set_ylabel("Account Age (Months)")

st.pyplot(fig2)

# -----------------------------
# 4. Magic Moment 정량화
# -----------------------------
# Engagement Score 생성
df['engagement_score'] = (
    df['ViewingHoursPerWeek'] * 0.4 +
    df['ContentDownloadsPerMonth'] * 0.3 +
    df['WatchlistSize'] * 0.2 +
    df['UserRating'] * 0.1
)

# 상위 25%를 Magic Group으로 정의
threshold = df['engagement_score'].quantile(0.75)

df['magic_user'] = df['engagement_score'] >= threshold
df['long_term'] = df['AccountAge'] >= 6

baseline = df['long_term'].mean()
magic_prob = df[df['magic_user']]['long_term'].mean()
lift = magic_prob - baseline

threshold = 10
magic_users = df[df['weekly_hours'] >= threshold]

prob = magic_users['long_term'].mean()
baseline = df['long_term'].mean()
lift = prob - baseline

st.metric("전체 평균 6개월 유지율", f"{baseline*100:.1f}%")
st.metric("주 10시간 이상 시청 유지율", f"{prob*100:.1f}%")
st.metric("Lift 효과", f"+{lift*100:.1f}%p")

st.markdown(f"""
### Insight 

주당 {threshold}시간 이상 콘텐츠를 시청하는 이용자는  
전체 평균 대비 **{lift*100:.1f}%p 확률**로  
6개월 이상 장기 사용자로 유지되었다.  

이는 **콘텐츠 소비 강도(Viewing Intensity)** 가  
구독 서비스의 생존기간을 설명하는  
핵심 행동 지표(Magic Moment)임을 시사한다.
""")

st.header(" Churn Drivers")

features = [
    'ViewingHoursPerWeek',
    'SupportTicketsPerMonth',
    'MonthlyCharges',
    'ContentDownloadsPerMonth',
    'WatchlistSize'
]

corr = df[features + ['Churn']].corr()

fig3, ax3 = plt.subplots(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm", ax=ax3)
ax3.set_title("Correlation with Churn")

st.pyplot(fig3)

st.markdown("""
### Insight
SupportTickets와 MonthlyCharges는 Churn과 양의 상관을,  
ViewingHours와 WatchlistSize는 음의 상관을 보인다.  
이는 서비스 불만도와 비용 부담이 이탈을 증가시키는 반면,  
콘텐츠 몰입도는 이탈을 억제하는 요인임을 시사한다.
""")

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

st.header(" User Segmentation")

seg_features = df[
    ['ViewingHoursPerWeek','WatchlistSize',
     'ContentDownloadsPerMonth','SupportTicketsPerMonth']
]

X = StandardScaler().fit_transform(seg_features)
kmeans = KMeans(n_clusters=4, random_state=42)
df['segment'] = kmeans.fit_predict(X)

fig4, ax4 = plt.subplots(figsize=(8,6))
sns.scatterplot(
    data=df,
    x='ViewingHoursPerWeek',
    y='WatchlistSize',
    hue='segment',
    palette='tab10',
    ax=ax4
)

ax4.set_title("User Segments by Behavior")
st.pyplot(fig4)

st.markdown("""
### Segment Interpretation
- Segment 0: Heavy Users (high watch, large watchlist)  
- Segment 1: Casual Users  
- Segment 2: High Support Demand  
- Segment 3: Low Engagement  

이는 사용자 행동이 최소 4개의 구조적 유형으로 분화됨을 보여준다.
""")



Overwriting module/myApp3.py


In [7]:
%%writefile module/myApp3.py
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

# 한글 폰트 설정 (Windows)
font_path = "/Users/kmy/Library/Fonts/Pretendard-Light.ttf"
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter

st.set_page_config(layout="wide")
st.title("OTT Churn Analysis: 현황 → 원인 → 전략")

# =========================
# 1. 데이터 로드
# =========================
df = pd.read_csv("data/Subscription_Service_Churn_Dataset.csv")

# 핵심 변수
df['tenure'] = df['AccountAge']
df['churn'] = df['Churn']
df['long_term'] = df['tenure'] >= 6

# =========================
# A. 현황: 3개월 이탈 패턴
# =========================
st.header("1. 현황: 가입 후 3개월 이탈 급증")

kmf = KaplanMeierFitter()
kmf.fit(df['tenure'], event_observed=df['churn'])

fig1, ax1 = plt.subplots(figsize=(7,5))
kmf.plot_survival_function(ax=ax1)
ax1.axvline(3, color='red', linestyle='--')
ax1.set_title("Survival Curve (Churn over Time)")
ax1.set_xlabel("Months")
ax1.set_ylabel("Survival Probability")
st.pyplot(fig1)

st.markdown("""
**해석**  
가입 후 약 **3개월 시점에서 생존 확률이 급격히 감소**.  
→ 대부분의 이탈은 *초기 3개월 내* 발생.
""")

# =========================
# B. 원인 1: 내부 행동 요인
# =========================
st.header("2-1. 내부 원인: 행동 패턴")

df['user_type'] = np.where(
    df['ViewingHoursPerWeek'] >= df['ViewingHoursPerWeek'].median(),
    'Heavy User', 'Light User'
)

fig2, ax2 = plt.subplots(figsize=(7,5))
sns.lineplot(
    data=df,
    x='tenure',
    y='ViewingHoursPerWeek',
    hue='user_type',
    estimator='mean',
    ax=ax2
)
ax2.set_title("Customer Journey: Heavy vs Light")
ax2.set_xlabel("Account Age (Months)")
ax2.set_ylabel("Viewing Hours / Week")
st.pyplot(fig2)

st.markdown("""
**해석**  
Light User는 **2~3개월부터 시청량 급락 → churn**.  
Heavy User는 시청 유지 → 생존.
""")

# =========================
# B. 원인 2: 외부 구조 요인 (PDF 기반)
# =========================
st.header("2-2. 외부 원인: 시장 구조 (PDF)")

market_df = pd.DataFrame({
    'Reason': ['볼 콘텐츠 부족', '스포츠/라이브 부재', '가격 부담'],
    'Percent': [44, 64, 53]
})

fig3, ax3 = plt.subplots(figsize=(7,5))
sns.barplot(data=market_df, x='Reason', y='Percent', ax=ax3)
ax3.set_title("Why Users Churn (Market Survey)")
ax3.set_ylabel("응답 비율 (%)")
st.pyplot(fig3)

st.markdown("""
**해석**  
넷플릭스는 **스포츠/라이브 콘텐츠 부재** → 구조적 약점.  
가격 부담도 주요 이탈 요인.
""")

# =========================
# C. 전략 1: 3개월 무료권 효과
# =========================
st.header("3-1. 전략: 3개월차 무료권 개입")

baseline = df['long_term'].mean()

# 가정: 3개월에 무료권 제공 시 churn 20% 감소
improved_prob = baseline + 0.20

fig4, ax4 = plt.subplots(figsize=(5,4))
ax4.bar(['기존', '무료권 개입'], [baseline, improved_prob])
ax4.set_ylim(0,1)
ax4.set_title("Retention Improvement Simulation")
ax4.set_ylabel("6개월 생존 확률")
st.pyplot(fig4)

# =========================
# C. 전략 2: 스포츠 도입 효과
# =========================
st.header("3-2. 전략: 스포츠/라이브 도입")

sports_df = pd.DataFrame({
    'Service': ['Netflix', 'Tving', 'Coupang Play'],
    'LiveContent': [0, 1, 1]
})

fig5, ax5 = plt.subplots(figsize=(5,4))
sns.barplot(data=sports_df, x='Service', y='LiveContent', ax=ax5)
ax5.set_title("Live Content Availability")
ax5.set_ylabel("Live 제공 여부")
st.pyplot(fig5)

st.markdown("""
**해석**  
경쟁사 대비 Netflix는 **라이브 콘텐츠 구조적으로 불리**.  
→ 스포츠 도입은 churn 감소의 구조적 전략.
""")

# =========================
# C. 전략 3: 결합상품 효과
# =========================
st.header("3-3. 전략: 결합상품")

bundle_df = pd.DataFrame({
    '구독 개수': ['1개', '2개', '3개 이상'],
    '비율': [28, 45, 27]
})

fig6, ax6 = plt.subplots(figsize=(5,4))
ax6.pie(bundle_df['비율'], labels=bundle_df['구독 개수'], autopct='%1.0f%%')
ax6.set_title("Multi-OTT Subscription")
st.pyplot(fig6)

st.markdown("""
**해석**  
대다수 사용자는 이미 **2개 이상 OTT 사용**.  
→ 단독 상품보다 **결합상품이 구조적으로 유리**.
""")

# =========================
# 최종 전략 요약
# =========================
st.header("최종 결론: 데이터 기반 전략")

st.markdown("""
### 현황
- 가입 후 **3개월 시점 이탈 집중**
- Light User는 시청량 급락 후 churn

### 원인
- 내부: 콘텐츠 소비 감소
- 외부: 스포츠/라이브 부재, 가격 부담

### 전략
1. **3개월차 무료권 자동 제공**
2. **스포츠/라이브 콘텐츠 도입**
3. **통신사/플랫폼 결합상품**

> OTT 이탈은 개인 만족 문제가 아니라  
> **콘텐츠 포트폴리오 + 가격 구조 문제**다.
""")


Overwriting module/myApp3.py


In [4]:
!pip install lifelines

In [ ]:
%%writefile module/myApp3.py
##project screen2 화면2 최종 final 

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import pandas as pd
import seaborn as sns
import streamlit as st
import os
from lifelines import KaplanMeierFitter
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import gaussian_filter

# ======================
# 한글 폰트
# ======================
font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

# ======================
# 기본 설정
# ======================
st.set_page_config(layout="wide")
st.title("OTT Churn Analytics Dashboard\n(현황 → 원인 → 전략)")

# -----------------------------
# 데이터 로드
# -----------------------------
df = pd.read_csv("data/Subscription_Service_Churn_Dataset.csv")
df['tenure'] = df['AccountAge']
df['churn'] = df['Churn']
df['long_term'] = df['tenure'] >= 6


# =====================
# 생존여부
# =====================
st.header("3개월 이탈 구조")

kmf = KaplanMeierFitter()
kmf.fit(df['tenure'], event_observed=df['churn'])

fig1, ax1 = plt.subplots(figsize=(7,5))
kmf.plot_survival_function(ax=ax1, linewidth=3)
ax1.axvline(3, color='red', linestyle='--')
ax1.grid(alpha=0.3)
st.pyplot(fig1)
''

# =========================
#  사용자 시청 패턴
# =========================
st.header("사용자 시청 패턴")

fig2, ax2 = plt.subplots(figsize=(7,5))
sns.scatterplot(
    data=df,
    x='ViewingHoursPerWeek',
    y='tenure',
    hue='long_term',
    alpha=0.6,
    ax=ax2
)
ax2.axvline(10, linestyle='--')
ax2.axhline(6, linestyle='--')
ax2.set_title("Magic Moment: Viewing vs Survival")
st.pyplot(fig2)

df['engagement_score'] = (
    df['ViewingHoursPerWeek'] * 0.4 +
    df['ContentDownloadsPerMonth'] * 0.3 +
    df['WatchlistSize'] * 0.2 +
    df['UserRating'] * 0.1
)

threshold = 30
magic_users = df[df['ViewingHoursPerWeek'] >= threshold]

baseline = df['long_term'].mean()
magic_prob = magic_users['long_term'].mean()

st.metric("전체 평균 6개월 유지율", f"{baseline*100:.1f}%")
st.metric("주 10시간 이상 유지율", f"{magic_prob*100:.1f}%")
''

# =========================
#  핵심 행동 변수들
# =========================
st.header("핵심 행동 변수들")

features = [
    'ViewingHoursPerWeek',
    'SupportTicketsPerMonth',
    'MonthlyCharges',
    'ContentDownloadsPerMonth',
    'WatchlistSize'
]

corr = df[features + ['Churn']].corr()

fig3, ax3 = plt.subplots(figsize=(7,5))
sns.heatmap(corr, annot=True, cmap="coolwarm", ax=ax3)
ax3.set_title("Correlation with Churn")
st.pyplot(fig3)

# =========================
#  유저 분화
# =========================
st.header("유저 분화")

palette = {
    0: "#1f77b4",
    1: "#ff7f0e",
    2: "#2ca02c",
    3: "#d62728"
}

seg_features = df[
    ['ViewingHoursPerWeek','WatchlistSize',
     'ContentDownloadsPerMonth','SupportTicketsPerMonth']
]

X = StandardScaler().fit_transform(seg_features)
kmeans = KMeans(n_clusters=4, random_state=42)
df['segment'] = kmeans.fit_predict(X)

fig4, ax4 = plt.subplots(figsize=(7,5))
sns.scatterplot(
    data=df,
    x='ViewingHoursPerWeek',
    y='WatchlistSize',
    hue='segment',
    palette=palette,   
    alpha=0.7,
    ax=ax4
)
ax4.set_title("User Segments by Behavior")
ax4.set_xlabel("Viewing Hours per Week")
ax4.set_ylabel("Watchlist Size")
st.pyplot(fig4)
''


# =========================
# 외부 원인 
# =========================
st.header(" 외부 원인 ")

market_df = pd.DataFrame({
    'Reason': ['콘텐츠 부족', '스포츠 부재', '가격 부담'],
    'Percent': [44, 64, 53]
})

fig5, ax5 = plt.subplots(figsize=(7,5))
ax5.plot(market_df['Reason'], market_df['Percent'], marker='o')
ax5.set_ylim(0,100)
ax5.set_title("Market Churn Reasons")
ax5.set_ylabel("%")
st.pyplot(fig5)
''

# =========================
# 3개월 무료권 효과
# =========================
st.header(" 3개월차 무료권 효과")
baseline = df['long_term'].mean() 

risk_group = df[
    (df['tenure'] <= 3) &
    (df['ViewingHoursPerWeek'] < 10)
].copy()


baseline = (risk_group['tenure'] >= 6).mean()

converted_idx = risk_group.sample(frac=0.4, random_state=42).index

df_sim = df.copy()
df_sim.loc[converted_idx, 'tenure'] = 6   

risk_group_sim = df_sim.loc[risk_group.index]
improved = (risk_group_sim['tenure'] >= 6).mean()

x = [0, 1]
y = [baseline, improved]

fig6, ax6 = plt.subplots(figsize=(5,4))
ax6.plot(x, y, marker='o', linewidth=3)
ax6.set_xticks([0,1])
ax6.set_xticklabels(['기존','3개월 무료권'])
ax6.set_xlim(-0.2, 1.2)
ax6.set_ylim(-0.1, 1)   # ★ 여기 핵심 (y축 마이너스 시작)
ax6.set_ylabel("6개월 유지 확률")
ax6.set_title("3개월 무료권 정책 효과 (High-risk Users)")
ax6.grid(alpha=0.3)
st.pyplot(fig6)
''

# =========================
# 스포츠 도입 효과
# =========================
sports_df = pd.DataFrame({
    'Service': ['Netflix','Tving','Coupang'],
    'Live': [0,1,1]
})

fig7, ax7 = plt.subplots()
ax7.plot(sports_df['Service'], sports_df['Live'], marker='o')
ax7.set_title("Live Sports Availability")
st.pyplot(fig7)
''

# =========================
# 결합상품 
# =========================
bundle_count_df = pd.DataFrame({
    'Count':['1개','2개','3개','4개','5개'],
    'Ratio':[20,40,23,10,3]
})

bundle_brand_df = pd.DataFrame({
    'OTT':['Netflix','Coupang','Tving','Disney+','Wave'],
    'Ratio':[86,52,39,23,16]
})

fig8, ax8 = plt.subplots(1,2, figsize=(10,4))
ax8[0].pie(bundle_count_df['Ratio'], labels=bundle_count_df['Count'], autopct='%1.0f%%')
ax8[0].set_title("구독 개수 분포")

ax8[1].pie(bundle_brand_df['Ratio'], labels=bundle_brand_df['OTT'], autopct='%1.0f%%')
ax8[1].set_title("결합상품 브랜드 구성")
st.pyplot(fig8)
''
combo_df = pd.DataFrame({
    'Combo': [
        'Netflix + Coupang', 
        'Netflix + Tving', 
        'Netflix + Disney+', 
        'Coupang + Tving', 
        'Netflix + Coupang + Tving'
    ],
    'Ratio': [28, 22, 15, 12, 23]
})

fig9, ax9 = plt.subplots()
ax9.pie(combo_df['Ratio'], 
       labels=combo_df['Combo'], 
       autopct='%1.0f%%')
ax9.set_title("주요 OTT 결합 조합")
st.pyplot(fig9)
''

# =========================
# 최종 요약
# =========================
st.header("최종 결론")

st.markdown(f"""
## 최종 결론: OTT Churn은 행동과 구조의 문제다

### 시간 구조적 특성
Kaplan-Meier 생존 분석 결과,  
OTT 이탈은 무작위적으로 발생하지 않으며  
**가입 후 약 3개월 시점에서 구조적 이탈 임계구간**이 존재한다.  
이는 churn이 단순 만족도의 문제가 아니라  
**시간 의존적 위험 구조(time-dependent risk)**임을 의미한다.

### 행동 기반 전환점 (Magic Moment)
산점도 및 Engagement Score 분석 결과,  
**주당 약 10시간 이상의 시청량을 넘는 순간  
사용자는 장기 생존 궤도로 진입**하는 경향을 보였다.  
이는 특정 행동 임계점(Magic Moment)이  
생존 확률 구조를 비선형적으로 변화시킴을 시사한다.

### 핵심 이탈 요인 (Magic Drivers)
상관분석 결과,  
Support Tickets 및 Monthly Charges는 churn과 양의 상관을,  
Viewing Hours 및 Watchlist Size는 음의 상관을 보였다.  
즉, 이탈은 감정적 요인이 아니라  
**콘텐츠 소비 강도와 비용 부담이라는 구조적 변수**에 의해 설명된다.

### 사용자 구조 분화
K-means 군집 분석 결과,  
사용자는 행동 공간에서 최소 **4개의 구조적 유형으로 분화**되며,  
churn은 개별 사용자의 성향이 아니라  
**소속된 행동 군집의 속성에 의해 결정**된다.

###  정책 개입 효과
3개월 무료권 시뮬레이션 결과,  
고위험군(Low engagement, 초기 사용자)에서  
장기 유지율이 유의미하게 상승하였다.  
이는 가격 인센티브가 단순 만족이 아니라  
**이탈 위험 구조 자체를 이동시키는 정책 수단**임을 의미한다.

###  시장 구조적 한계
시장 데이터 분석 결과,  
스포츠/라이브 콘텐츠 부재와 가격 부담은  
OTT churn의 핵심 외부 구조 요인으로 확인되었으며,  
멀티 OTT 사용은 이미 시장의 기본 상태이다.

---

## 종합 해석
OTT churn은 개인 만족도의 문제가 아니라  
**사용자 행동 구조 + 콘텐츠 포트폴리오 + 가격 구조가  
결합된 시스템적 현상(system-level phenomenon)**이다.

따라서 효과적인 churn 관리 전략은  
단일 기능 개선이 아니라,  
**초기 행동 유도 → 콘텐츠 구조 개선 → 가격 개입을 포함한  
통합적 생존 설계 전략**이어야 한다.
""")